# EXTRACCIÓN DE LOS DATOS DE LINKEDIN

Para la red social de Linkedin, se extraerán datos con dos scripts diferentes.<br>
- Script para sacar información de los perfiles e introducirlos en la BBDD
- Script para sacar información de los posts e introducirlos en la BBDD

## PSEUDOCÓDIGO

**El pseudocódigo del programa será de la siguiente manera**

Abrir Base de Datos
Seleccionar y extraer los datos de la tabla donde se almacenan los enlaces de los diferentes enlaces a los perfiles de Linkedin<br><br>
Extraer las cuentas de Linkedin<br><br>
Para cada una de las cuentas:<br>
   &emsp; extraer los datos de la cuentas<br>
   &emsp; introducir los datos en la tabla correspondiente<br><br>
   &emsp; Para cada uno de los mensajes de la cuenta:<br>
   &emsp; &emsp; extraer los datos de los mensajes<br>
   &emsp; &emsp; introducir los datos en la tabla correspondiente<br>

## LIBRERÍAS

Para la correcta extracción y recopilación de los datos, se aplicarán diferentes librerías pero la principal es:<br><br>

**__SELENIUM__**: Es un entorno de pruebas de software para aplicaciones basadas en la web. Su función principal es realizar web scraping de manera automatizada mediante diferentes métodos posibles como HTML, CSS, XPATH, etc..<br><br>Esto es gracias a la implantación de un driver que automatizará el navegador y que se puede descargar __[aquí](https://selenium-python.readthedocs.io/installation.html)__.<br><br>
La documentación de Selenium para Python se puede consultar __[aquí](https://selenium-python.readthedocs.io/getting-started.htmly)__.<br>

La librería selenium no es como otras librerías usadas en otras redes sociales que son especializadas en la extracción de datos en esa red social, selenium recorre la web y es el programador el que recoge los datos.<br>

Otras librerías que se usan son:
- **Pandas** : Manipulación de datos
- **Mysql.connector**: Conexión con la base de datos
- **Codecs**: Normalización de caracteres
- **Time**: Manipula y da información de la cantidad de tiempo
- **Datetime**: Facilita la manipulación de datos tipo fecha


## PASOS PREVIOS

Para realizar estos scripts, primeramente se añaden dos tablas a la BBDD con la siguiente sintaxis:

- **TABLA PARA PERFILES DE LINKEDIN**:<br>

CREATE TABLE IF NOT EXISTS perfiles_linkedin(<br>
    &emsp; Url VARCHAR(200),<br>
    &emsp; Tipo VARCHAR(15),<br>
    &emsp; Nombre VARCHAR(100),<br>
    &emsp; Seguidores BIGINT,<br>
	&emsp; Resumen TEXT,<br>
    &emsp; Sector VARCHAR(200),<br>
    &emsp; Tamaño VARCHAR(50),<br>
    &emsp; Empleados BIGINT,<br>
    &emsp; Sede VARCHAR(100),<br>
    &emsp; Fundacion BIGINT,<br>
    &emsp; Especialidades TEXT,<br>
    &emsp; Antiguos_empleados BIGINT,<br>
    &emsp; Donde_viven TEXT,<br>
    &emsp; Donde_trabajan TEXT,<br>
    &emsp; Que_hacen TEXT,<br>
    &emsp; Que_estudiaron TEXT,<br>
    &emsp; Aptitudes TEXT,<br>
    &emsp; Conexion TEXT<br>
)ENGINE = INNODB;<br>




- **TABLA PARA LOS POSTS DE LINKEDIN**:<br>

CREATE TABLE IF NOT EXISTS posts_linkedin(<br>
    &emsp; id INT,<br>
    &emsp; Url VARCHAR(200),<br>
    &emsp; Nombre VARCHAR(200),<br>
    &emsp; Fecha DATETIME,<br>
    &emsp; Texto TEXT,<br>
    &emsp; Videos INT,<br>
    &emsp; Fotos INT,<br>
    &emsp; Articulos INT,<br>
    &emsp; Reacciones INT,<br>
    &emsp; Comentarios INT,<br>
    &emsp; Compartido INT<br>
)ENGINE = INNODB;<br>




## CÓDIGO

Ya que se tiene todo los requisitos, ahora se abre la conexión con la Base de Datos y se empieza a codificar. <br>
Lo primero de todo es importar las librerias que necesitamos, serán varias.

In [ ]:
import pandas as pd
import mysql.connector
import codecs
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Acto seguido, se abre la conexión de la base de Datos con Python 

In [ ]:
database = mysql.connector.connect(
    host= "localhost",
    user= "root",
    passwd= "",
    database= "TFG",
    use_unicode = True, 
    charset="utf8mb4",
    collation = "utf8mb4_general_ci"
)
database.set_charset_collation("utf8mb4","utf8mb4_general_ci")

cursor= database.cursor()

Se extraen las cuentas de Linkedin de la tabla de la Base de Datos **"universidades"**

In [ ]:
datos = pd.read_sql("SELECT * FROM universidades",database)
cuentas_instagram = datos["Instagram"].tolist()
universidad = datos["Universidad"].tolist()
print(datos.info())

Como no todas las universidades tienen cuenta de linkedin, se pasará a limpiar la lista.

In [ ]:
for a in range(len(cuentas_linkedin)-1,-1,-1):
    if type(cuentas_linkedin[a]) != str:
        cuentas_linkedin.remove(cuentas_linkedin[a])
        universidad.remove(universidad[a])
print( len(cuentas_linkedin),len(universidad))

Empezamos creando el driver y las opciones del driver.

In [ ]:
opts= Options()
opts.add_argument("--user-agent=Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25")
opts.add_argument("--start-maximized")
opts.add_experimental_option('excludeSwitches', ['enable-logging'])
driver= webdriver.Chrome('./chromedriver.exe', options=opts)

Ahora pasamos a recoger la información de los perfiles con el siguiente pseudocódigo, más elaborado:<br>
    Registro en Linkedin.<br>
    para cada universidad:<br>
        recoger información basica<br>
        recoger información acerca de<br>
        recoger datos antiguos empleados<br>
        meter datos en base de datos<br>

Acto seguido se pasa a elaborar el bucle que recogerá los datos de los perfiles

In [ ]:
## AUTENTIFICACIÓN
driver.get(cuentas_linkedin[0])
driver.implicitly_wait(3)
driver.find_element_by_css_selector('button.promo__dismiss').click()
try:
    driver.find_element(By.XPATH, "//a[@class='nav__button-secondary']").click()
    usuario =  driver.find_element(By.XPATH, '//input[@name="session_key"]')
    usuario.send_keys("100383702@alumnos.uc3m.es")
    password = driver.find_element(By.XPATH, '//input[@name="session_password"]')
    password.send_keys("PASS")
    driver.find_element(By.XPATH, "//button[@class='btn__primary--large from__button--floating']").click()
except:
    driver.find_element_by_css_selector('button.authwall-join-form__form-toggle--bottom.form-toggle').click()
    usuario =  driver.find_element(By.XPATH, '//input[@name="session_key"]')
    usuario.send_keys("100383702@alumnos.uc3m.es")
    password = driver.find_element(By.XPATH, '//input[@name="session_password"]')
    password.send_keys("C3$999UniLINK")
    driver.find_element(By.XPATH, "//button[@class='sign-in-form__submit-button']").click()

for a in range(37,len(cuentas_linkedin)): #ESTANDARIZACIÓN ENLACES
    driver.get(cuentas_linkedin[a])re
    tipo = cuentas_linkedin[a].index("/",10)
    if cuentas_linkedin[a][tipo+1] == "i" :
        continue
    href = cuentas_linkedin[a][cuentas_linkedin[0].index("/",10):cuentas_linkedin[a].rfind("/")+1]


    ## Recoger aqui info del primer cuadrado, antes del acerca de.
    nombre = driver.find_element(By.XPATH, "//h1/span").text
    print(nombre)
    seguidores = driver.find_elements(By.XPATH, "//div[@class='org-top-card-summary-info-list__info-item']")[-1].text
    try:
        seguidores = int(seguidores[:seguidores.index(" ")])
    except:
        seguidores = seguidores[:seguidores.index(" ")]
        seguidres = int(seguidores.replace(".",""))



    ## Recoger datos pestaña de ACERCA DE

    acerca_de = f"//a[@href='{href}about/']"
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH, acerca_de ))).click()
    n = 0
    resumen = driver.find_element_by_css_selector("p.break-words.white-space-pre-wrap.mb5.text-body-small.t-black--light").text

    tel = driver.find_elements_by_css_selector("dd.mb4.text-body-small.t-black--light")[1].text
    if tel[-1].isnumeric():
        pass
    else:
        n = n-1

    sector = driver.find_elements_by_css_selector("dd.mb4.text-body-small.t-black--light")[n+2].text
    tamano = driver.find_element_by_css_selector("dd.mb1.text-body-small.t-black--light").text
    empleados = driver.find_elements_by_css_selector("dd.mb4.text-body-small.t-black--light")[n+3].text
    try:
        empleados = int(empleados[:empleados.index(" ")])
    except:
        empleados = empleados[:empleados.index(" ")]
        empleados = int(empleados.replace(".",""))
    try:
        sede = driver.find_elements_by_css_selector("dd.mb4.text-body-small.t-black--light")[n+4].text
    except:
        sede = None
    try:
        fundacion = int(driver.find_elements_by_css_selector("dd.mb4.text-body-small.t-black--light")[n+5].text)
    except:
        fundacion = None
    try:
        especialidades =  driver.find_elements_by_css_selector("dd.mb4.text-body-small.t-black--light")[n+6].text
    except:
        especialidades = None
    datos = [cuentas_linkedin[a],"school",nombre,seguidores,resumen,sector,tamano,empleados,sede,fundacion,especialidades]

    
    ## recoger datos de antiguos empleados

    empleados = f"//a[@href='{href}people/']"
    try:
        WebDriverWait(driver,7).until(EC.presence_of_element_located((By.XPATH, empleados ))).click()
    except:
        pass
    sleep(3)
    antiguos_alumnos=  driver.find_element_by_xpath( "//div[@class='display-flex full-width justify-space-between align-items-center pt5 ph5']//span" ).get_attribute("textContent").strip()
    try:
        antiguos_alumnos = int(antiguos_alumnos[:antiguos_alumnos.find("\xa0")])
    except:
        antiguos_alumnos = antiguos_alumnos[:antiguos_alumnos.find("\xa0")]
        antiguos_alumnos = int(antiguos_alumnos.replace(".",""))
    datos.append(antiguos_alumnos)
    
    if antiguos_alumnos <1:
        datos.extend([None,None,None,None,None,None])
    else:
        driver.find_element_by_xpath("//button[@class='org-people__show-more-button t-16 t-16--open t-black--light t-bold']").click()
        siguiente = True
        while siguiente:
            carrouseles = driver.find_elements_by_xpath( "//div[@class='insight-container']")
            for lista  in carrouseles:
                carro = ""
                titulo = lista.find_element(By.XPATH,".//h4").text
                if titulo == "":
                    continue
                numeros = lista.find_elements_by_xpath(".//strong")
                textos = lista.find_elements(By.XPATH,".//span[@class='org-people-bar-graph-element__category']")
                for i in range(len(numeros)):
                    carro = carro + f"{numeros[i].text} | {textos[i].text} $ "
                carro = carro[:-2]
                datos.append(carro)
            try:
                driver.find_element_by_xpath("//button[@class='artdeco-pagination__button artdeco-pagination__button--next artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--tertiary ember-view']").click()
                sleep(1)
            except:
                siguiente = False
        #driver.find_element_by_css_selector('button.org-people__show-more-button t-16 t-16--open t-black--light t-bold').click()

    ##meter los datos a la base de datos
    datos = tuple(datos)
    for dato in datos:
        print(dato)
    cursor.execute("INSERT INTO perfiles_linkedin VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",datos)
    database.commit()

Despues de extraer los perfiles, vamos a extraer los post, con el siguiente pseudocódigo:<br>

Registro en Linkedin.<br>
    para cada universidad:<br>
        recoger información basica<br>
        recoger información post<br>
        texto, videos, imagenes, datos, comentarios, reacciones<br>
        meter datos en base de datos<br>

In [ ]:
driver.get(cuentas_linkedin[0])
driver.implicitly_wait(3)
driver.find_element_by_css_selector('button.promo__dismiss').click()
try:#diferentes maneras de hacer el loggin
    driver.find_element(By.XPATH, "//a[@class='nav__button-secondary']").click()
    usuario =  driver.find_element(By.XPATH, '//input[@name="session_key"]')
    usuario.send_keys("100383702@alumnos.uc3m.es")
    password = driver.find_element(By.XPATH, '//input[@name="session_password"]')
    password.send_keys("C3$999UniLINK")
    driver.find_element(By.XPATH, "//button[@class='btn__primary--large from__button--floating']").click()
except:
    driver.find_element_by_css_selector('button.authwall-join-form__form-toggle--bottom.form-toggle').click()
    usuario =  driver.find_element(By.XPATH, '//input[@name="session_key"]')
    usuario.send_keys("100383702@alumnos.uc3m.es")
    password = driver.find_element(By.XPATH, '//input[@name="session_password"]')
    password.send_keys("C3$999UniLINK")
    driver.find_element(By.XPATH, "//button[@class='sign-in-form__submit-button']").click()
a = 0
sleep(12)
for a in range(59,len(cuentas_linkedin)):# estandarización de enlaces
    driver.get(cuentas_linkedin[a])
    tipo = cuentas_linkedin[a].index("/",10)
    if cuentas_linkedin[a][tipo+1] == "i" :
        pass
    href = cuentas_linkedin[a][cuentas_linkedin[0].index("/",10):cuentas_linkedin[a].rfind("/")+1]

    ## Recoger aqui info del primer cuadrado, antes del acerca de.
    nombre = driver.find_element(By.XPATH, "//h1/span").text
    print(nombre)

    posts = f"//a[@href='{href}posts/']"
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH, posts ))).click()
    screen_height = driver.execute_script("return window.screen.height;")
    sleep(3)
    i = 1
    while True:
        # scroll lo qeu mide la pantalla
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        sleep(1)
        # actualiza la pagina para nuevo scroll
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # rompe si al actualizar no se actualizan las dimensiones
        if (screen_height) * i > scroll_height:
            break
    sleep(1)
    posts= driver.find_elements_by_xpath("//div[@class='scaffold-finite-scroll__content']/div")
    print(len(posts))
    for b, post in enumerate(posts):#recogemos post
        ## recogemos fecha
        try:
            fecha = post.find_element_by_css_selector("span.feed-shared-actor__sub-description.t-12.t-normal.t-normal.t-black--light").get_attribute("textContent").strip()
            fecha = fecha[:fecha.index(" ",4)]
            datos_fecha = fecha.split()
            if datos_fecha[-1][0] == "d":
                fecha = date.today() + relativedelta(days = -int(datos_fecha[0]))
            elif datos_fecha[-1][0] == "s":
                fecha =  date.today() + relativedelta(months = -int(datos_fecha[0]))
            else:
                fecha = date.today() + relativedelta(years = -int(datos_fecha[0]))
            
        except:
            continue
        ## recogemos texto
        try:
            texto = post.find_element_by_css_selector("span.break-words").get_attribute("textContent").strip()
            
        except:
            texto = None
        try:
            reacciones = post.find_element_by_css_selector("span.social-details-social-counts__reactions-count").get_attribute("textContent").strip()
        except:
            reacciones = 0
        
        interacciones = post.find_elements_by_css_selector("li.social-details-social-counts__item.social-details-social-counts__item--with-social-proof")
        if len(interacciones)== 0:
            comentarios = 0
            compartido = 0
        else:
            comentarios = 0
            compartido = 0
            for interaccion in interacciones:
                inter = interaccion.get_attribute("textContent").strip()
                if "comen" in inter:
                    comentarios = int(inter[:inter.index(" ")])
                elif "compart" in inter:
                    compartido = int(inter[:inter.index(" ")])
                
        ## recogemos resto de variables 
        videos = len(post.find_elements_by_xpath(".//div[@class='external-video-viewer']"))
        imagenes = len(post.find_elements_by_xpath(".//div[@class = 'feed-shared-image__container']"))
        lecturas = len(post.find_elements_by_xpath(".//article[@class ='feed-shared-article feed-shared-update-v2__content']"))
        datos =(a,cuentas_linkedin[a],nombre,fecha,texto,videos,imagenes,lecturas,reacciones,comentarios,compartido)
        cursor.execute("INSERT INTO posts_linkedin VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",datos)
        database.commit()

### Con este pequeño script están los datos de la red social Linkedin extraidos.<br><br> Duración aproximada 15h